# Revista de Gestão e Contabilidade da UFPI

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(query, url_search_text):
    url_search = url_search_text.replace('__query__', query)
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links



In [4]:
url_search_text = 'https://revistas.ufpi.br/index.php/gecont/search/search?query=__query__&authors=&title=&abstract=&galleyFullText=&suppFiles=&dateFromMonth=&dateFromDay=&dateFromYear=2014&dateToMonth=&dateToDay=&dateToYear=2021&dateToHour=23&dateToMinute=59&dateToSecond=59&discipline=&subject=&type=&coverage=&indexTerms='

links = listar_links_pagina('contabilidade', url_search_text)

links[0]

'https://revistas.ufpi.br/index.php/gecont/article/view/6292'

In [5]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [6]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'articleTitle'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [7]:
def captura_ano(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [8]:
def captura_resumo(soup):
    try:
        abstract_tag = soup.find('div', {'id': 'articleAbstract'})
        abstract = abstract_tag.find('div').text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [9]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'id': 'articleSubject'})
        kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [10]:
def captura_autores(soup):
    autores = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        
        for p in authors_tag:
            try:
                autor = p.find('em').text
            except:
                autor = ''
            autores.append(autor)
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [11]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [12]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Gestão e Contabilidade da UFPI'}
    url_search_text = 'https://revistas.ufpi.br/index.php/gecont/search/search?query=__query__&authors=&title=&abstract=&galleyFullText=&suppFiles=&dateFromMonth=&dateFromDay=&dateFromYear=2014&dateToMonth=&dateToDay=&dateToYear=2021&dateToHour=23&dateToMinute=59&dateToSecond=59&discipline=&subject=&type=&coverage=&indexTerms='

    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links_pagina(query, url_search_text)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_ano(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [13]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2016-01-01,EXECUÇÃO ORÇAMENTÁRIA DE NATUREZA AMBIENTAL NO...,Este estudo teve o objetivo de examinar se mun...,Exploração Mineral. Despesas Orçamentárias Amb...,3,Valtemar Soares de Melo,Francisco Marton Gleuson Pinheiro,Raimundo Nonato Lima Filho,,,,Graduado em Ciências Contábeis pela Universida...,Doutorando em Administração pela Universidade ...,Doutor em Administração pela Universidade Fede...,,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
1,2014-01-01,Comparação do nível de conhecimento sobre Cont...,A Universidade Federal da Bahia (UFBA) forma u...,Contabilidade Social e Ambiental. Ensino da Co...,5,Mirian Gomes Conceição,Nverson da Cruz Oliveira,Sonia Maria da Silva Gomes,Neylane dos Santos Oliveira,Francisco Gaudêncio,,Universidade Federal da Bahia,,,,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
2,2016-01-01,A APLICAÇÃO DO MANUAL DE CONTABILIDADE APLICAD...,Este artigo teve por objetivo verificar se a P...,Município. Contabilidade Patrimonial Govername...,3,Abimael de Jesus Barros Costa,Mizael Vieira Nunes,André Porfírio de Almeida,,,,Brasil,,,,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI


In [14]:
df.to_csv('../data/RGC_UFPI.csv', index=False)

In [15]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2016-01-01,EXECUÇÃO ORÇAMENTÁRIA DE NATUREZA AMBIENTAL NO...,Este estudo teve o objetivo de examinar se mun...,Exploração Mineral. Despesas Orçamentárias Amb...,3,Valtemar Soares de Melo,Francisco Marton Gleuson Pinheiro,Raimundo Nonato Lima Filho,,,,Graduado em Ciências Contábeis pela Universida...,Doutorando em Administração pela Universidade ...,Doutor em Administração pela Universidade Fede...,,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
1,2014-01-01,Comparação do nível de conhecimento sobre Cont...,A Universidade Federal da Bahia (UFBA) forma u...,Contabilidade Social e Ambiental. Ensino da Co...,5,Mirian Gomes Conceição,Nverson da Cruz Oliveira,Sonia Maria da Silva Gomes,Neylane dos Santos Oliveira,Francisco Gaudêncio,,Universidade Federal da Bahia,,,,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
2,2016-01-01,A APLICAÇÃO DO MANUAL DE CONTABILIDADE APLICAD...,Este artigo teve por objetivo verificar se a P...,Município. Contabilidade Patrimonial Govername...,3,Abimael de Jesus Barros Costa,Mizael Vieira Nunes,André Porfírio de Almeida,,,,Brasil,,,,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
3,2015-01-01,EDUCAÇÃO CONTÁBIL EM TEMPOS DE IFRS: Estudo de...,"Com globalização, o avanço tecnológico e a ado...",Educação. Contabilidade. Justus-Liebig-Univers...,3,Raquel Luz de Lima,Abimael de Jesus Barros Costa,Cláudio Moreira Santana,,,,Universidade de Brasília,Universidade de Brasília,Universidade Federal de Goiais - UFG,,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
4,2019-01-01,GRI: A PERCEPÇÃO DE RELEVÂNCIA POR ONGs FLUMIN...,É percebida uma preocupação crescente do homem...,Sustentabilidade. GRI. Organizações Não Govern...,4,Vinicius Fasuolo Trancoso,José Ricardo Maia de Siqueira,Monica Zaidan Gomes,Isabel Cabral,,,UFRJ - Universidade Federal do Rio de Janeiro,,URFJ - Universidade Federal do Rio de Janeiro,URFJ - Universidade Federal do Rio de Janeiro\...,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
5,2016-01-01,VALORES ORGANIZACIONAIS E SUPORTE SOCIAL NO TR...,A pesquisa objetiva identificar os valoresorga...,Valores Organizacionais. Suporte Social. Compo...,5,James Jonatas da Silva,Alexandre de Freitas Carneiro,Flávio de São Pedro Filho,Douglas Vidal,Deyvison Lima de Oliveira,,Universidade Federal de Rondônia,Universidade Federal de Rondônia,Universidade Federal de Rondônia,Universidade Guarulhos,Universidade Federal de Rondônia,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
6,2019-01-01,GRI: A PERCEPÇÃO DE RELEVÂNCIA POR ONGs FLUMIN...,É percebida uma preocupação crescente do homem...,Sustentabilidade. GRI. Organizações Não Govern...,4,Vinicius Fasuolo Trancoso,José Ricardo Maia de Siqueira,Monica Zaidan Gomes,Isabel Cabral,,,UFRJ - Universidade Federal do Rio de Janeiro,,URFJ - Universidade Federal do Rio de Janeiro,URFJ - Universidade Federal do Rio de Janeiro\...,,,https://revistas.ufpi.br/index.php/gecont/arti...,Revista de Gestão e Contabilidade da UFPI
